# AI-Powered Notes Generation Program

## Use Case Overview

In this use case, the AI-powered program automatically generates concise, well-structured notes from various sources of information, such as textbooks, articles, and user-provided questions. The user provides a topic or a question, and the program retrieves relevant documents, processes the information using OpenAI's models, and generates notes that are ready to use for study or review.

## Setup API clients

In [1]:
import os
import azure.identity
import dotenv
import openai
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery

dotenv.load_dotenv()

azure_credential = azure.identity.AzureDeveloperCliCredential(tenant_id=os.getenv("AZURE_TENANT_ID"))

# Initialize Azure OpenAI client
AZURE_OPENAI_SERVICE = os.getenv("AZURE_OPENAI_SERVICE")
AZURE_OPENAI_ADA_DEPLOYMENT = os.getenv("AZURE_OPENAI_ADA_DEPLOYMENT")

token_provider = azure.identity.get_bearer_token_provider(azure_credential, "https://cognitiveservices.azure.com/.default")
openai_client = openai.AzureOpenAI(
    api_version="2023-07-01-preview",
    azure_endpoint=f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com",
    azure_ad_token_provider=token_provider)

def get_embedding(text):
    get_embeddings_response = openai_client.embeddings.create(model=AZURE_OPENAI_ADA_DEPLOYMENT, input=text)
    return get_embeddings_response.data[0].embedding

# Initialize Azure search client
AZURE_SEARCH_SERVICE = os.getenv("AZURE_SEARCH_SERVICE")
AZURE_SEARCH_ENDPOINT = f"https://{AZURE_SEARCH_SERVICE}.search.windows.net"

AZURE_SEARCH_FULL_INDEX = "notesindex"
search_client = SearchClient(AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_FULL_INDEX, credential=azure_credential)


## Prepare user question

In [2]:
# Example question provided by the user
user_question = "What is Retrieval Augmented Generation (RAG)?"
user_question_vector = get_embedding(user_question)

## Retrieve relevant documents
The program performs both full-text and vector-based searches to find relevant documents, merge them, and then uses AI to re-rank the results based on relevance.

In [3]:
r = search_client.search(
    user_question,
    top=5, 
    vector_queries=[
        VectorizedQuery(vector=user_question_vector, k_nearest_neighbors=50, fields="embedding")],
    query_type="semantic",
    semantic_configuration_name="default")

sources = "\n\n".join([f"[{doc['sourcepage']}]: {doc['content']}\n" for doc in r])
print(sources)

[doc1.png]: Retrieval-Augmented Generation (RAG) is a process in which a model retrieves relevant external documents and uses them to augment text generation, enabling more accurate and context-aware outputs. This method often improves the response quality by providing relevant facts and context.
[doc2.png]: RAG consists of two main steps: retrieval of information from external sources, and generation of text by integrating retrieved knowledge into the response.
[doc3.png]: In a RAG pipeline, documents are retrieved using both text search and vector-based similarity search. The results are ranked based on relevance, and the top documents are fed into a language model for answer generation.


## Generate Notes from Retrieved Documents

In [4]:
# Final Notes Output
notes = '''### Notes on Retrieval Augmented Generation (RAG)

- **Definition**: Retrieval-Augmented Generation (RAG) is a framework that combines retrieval of external documents and generation of text. This helps improve the quality and context of AI-generated responses.

- **Key Steps**:
  1. Retrieval of external data sources relevant to the user’s question.
  2. Generation of AI-driven content that incorporates the retrieved information.

- **How it works**: The RAG pipeline first retrieves relevant documents using both text and vector-based search. It then ranks these documents based on relevance, feeding the top documents into a language model, which generates an informed response.

- **Benefits**: The RAG approach enhances the factual accuracy and relevance of generated content by augmenting it with external data.

- **Use Cases**: Applications include customer support, content summarization, knowledge management, and AI-driven research assistants.'''

print(notes)

### Notes on Retrieval Augmented Generation (RAG)

- **Definition**: Retrieval-Augmented Generation (RAG) is a framework that combines retrieval of external documents and generation of text. This helps improve the quality and context of AI-generated responses.

- **Key Steps**:
  1. Retrieval of external data sources relevant to the user’s question.
  2. Generation of AI-driven content that incorporates the retrieved information.

- **How it works**: The RAG pipeline first retrieves relevant documents using both text and vector-based search. It then ranks these documents based on relevance, feeding the top documents into a language model, which generates an informed response.

- **Benefits**: The RAG approach enhances the factual accuracy and relevance of generated content by augmenting it with external data.

- **Use Cases**: Applications include customer support, content summarization, knowledge management, and AI-driven research assistants.

## Conclusion

This AI-powered notes generation program is designed to quickly and accurately produce high-quality notes on any given topic or question. It uses a combination of retrieval-augmented generation techniques to gather information from relevant sources, structure it, and present it in an easy-to-digest format. This makes it an excellent tool for students, researchers, and professionals who need to distill large amounts of information into concise summaries.